In [1]:
import xtrack as xt
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import apertls

In [2]:
path_lines = Path('/Users/gebruiker/sps_simulations/injection_lines')
line = xt.Line.from_json(Path(path_lines, 'injection_thin_approx_ap_with_deferred_expressions.json'))
#line = xt.Line.from_json(Path(path_lines, 'sps_lhc_q20_rf_with_ap.json'))
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

tw_ref = line.twiss4d()

for name in line.element_names:
    if line[name].__class__.__name__.startswith('Limit') or line[name].__class__.__name__.startswith('Multipole'):
        #Dogleg in LSS5 for extraction
        if 'qd.51710' in name:
            line[name].shift_y = -5.273e-3
        elif 'qfa.51810' in name:
            line[name].shift_y = -17.369e-3
        elif 'qd.51910' in name:
            line[name].shift_y = -5.273e-3
        #Dogleg in LSS1 for injection
        elif 'qd.11710' in name:
            line[name].shift_x = -4.80e-3
        elif 'qfa.11810' in name:
            line[name].shift_x = -2.97e-3
        elif 'qda.11910' in name:
            line[name].shift_x = -4.80e-3
    
    elif line[name].__class__.__name__.startswith('ThinSliceQuadrupole'):
        line['qd.51710'].shift_y = -5.273e-3
        line['qfa.51810'].shift_y = -17.369e-3
        line['qd.51910'].shift_y = -5.273e-3
        line['qd.11710'].shift_x = -4.80e-3 #Hannes Thesis says qda.
        line['qf.11810'].shift_x = -2.97e-3 #Hannes Thesis says qfa.
        line['qda.11910'].shift_x = -4.80e-3

tw_before = line.twiss4d()
tw = line.twiss()

Loading line from dict:   0%|          | 0/31362 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
tt = line.get_table()
bend_elements = np.union1d(tt.rows['mba.*'].name, tt.rows['mbb.*'].name)

In [ ]:
ap_manager = apertls.ManageApertures(line)
associated_apertures = ap_manager.find_associated_apertures()

In [ ]:
bend_elements = np.union1d(tt.rows['mba.*'].name, tt.rows['mbb.*'].name)

for name in bend_elements:
    if hasattr(line[name], '_parent'):
        L = line[name]._parent.length
        R = 1/line[name]._parent.k0

ok


In [10]:
def sagitta_circle(dipole_length, dipole_bending_radius, dipole_s_start, s):
    t = np.acos((s - dipole_s_start)/dipole_bending_radius)
    R_prime = dipole_bending_radius - np.sqrt(dipole_bending_radius**2 - dipole_length**2/4)

    return dipole_bending_radius*np.sin(t) - (dipole_bending_radius-R_prime)

In [7]:
bend_elements[:30]

array(['mba.10030..0', 'mba.10030..0_aper_downstream',
       'mba.10030..0_aper_upstream', 'mba.10030..1',
       'mba.10030..1_aper_downstream', 'mba.10030..1_aper_upstream',
       'mba.10030..entry_map', 'mba.10030..entry_map_aper_downstream',
       'mba.10030..entry_map_aper_upstream', 'mba.10030..exit_map',
       'mba.10030..exit_map_aper_downstream',
       'mba.10030..exit_map_aper_upstream', 'mba.10030_entry',
       'mba.10030_entry_aper_downstream', 'mba.10030_entry_aper_upstream',
       'mba.10030_exit', 'mba.10030_exit_aper_downstream',
       'mba.10030_exit_aper_upstream', 'mba.10050..0',
       'mba.10050..0_aper_downstream', 'mba.10050..0_aper_upstream',
       'mba.10050..1', 'mba.10050..1_aper_downstream',
       'mba.10050..1_aper_upstream', 'mba.10050..entry_map',
       'mba.10050..entry_map_aper_downstream',
       'mba.10050..entry_map_aper_upstream', 'mba.10050..exit_map',
       'mba.10050..exit_map_aper_downstream',
       'mba.10050..exit_map_aper_upstrea

In [18]:
line.element_names.index('qd.41510..0')


17477

In [20]:
for name in line.element_names[17477-20:17477+20]:
    print(name)

lsd.41505..0
lsd.41505..0_aper_downstream
drift_lsd.41505..1
lsd.41505_exit_aper_upstream
lsd.41505_exit
lsd.41505_exit_aper_downstream
drift_1065
drift_mdv.41507..1
mdv.41507_aper_upstream
mdv.41507
mdv.41507_aper_downstream
drift_mdv.41507..2
drift_1066
bpv.41508
drift_1067
qd.41510_entry_aper_upstream
qd.41510_entry
qd.41510_entry_aper_downstream
drift_qd.41510..0
qd.41510..0_aper_upstream
qd.41510..0
qd.41510..0_aper_downstream
drift_qd.41510..1
qd.41510..1_aper_upstream
qd.41510..1
qd.41510..1_aper_downstream
drift_qd.41510..2
qd.41510..2_aper_upstream
qd.41510..2
qd.41510..2_aper_downstream
drift_qd.41510..3
qd.41510..3_aper_upstream
qd.41510..3
qd.41510..3_aper_downstream
drift_qd.41510..4
qd.41510..4_aper_upstream
qd.41510..4
qd.41510..4_aper_downstream
drift_qd.41510..5
qd.41510..5_aper_upstream


In [56]:
line['mba.10030_entry_aper_downstream']._parent

AttributeError: 'LimitRect' object has no attribute '_parent'

In [ ]:
def beam_sagitta_calc(R, theta, nb_points):
    

In [ ]:
plt.figure()
R = 3
theta = np.pi/4
s = np.arange(R*np.cos(-theta), R*np.cos(theta))
x = 
plt.plot()

In [46]:
tt.rows['mbb.10070.*']

Table: 18 rows, 11 cols
name                                             s element_type       isthick isreplica ...
mbb.10070_entry_aper_upstream               16.755 LimitRect            False     False
mbb.10070_entry                             16.755 Marker               False     False
mbb.10070_entry_aper_downstream             16.755 LimitRect            False     False
mbb.10070..entry_map_aper_upstream          16.755 LimitRect            False     False
mbb.10070..entry_map                        16.755 ThinSliceBendEntry   False     False
mbb.10070..entry_map_aper_downstream        16.755 LimitRect            False     False
mbb.10070..0_aper_upstream                 17.7983 LimitRect            False     False
mbb.10070..0                               17.7983 ThinSliceBend        False     False
mbb.10070..0_aper_downstream               17.7983 LimitRect            False     False
mbb.10070..1_aper_upstream                 21.9717 LimitRect            False     False
mbb.

In [48]:
line['mbb.10070..0']._parent

Bend(length=6.26, k0=0.00135, k1=0, h=0.00135, k0_from_h=False, model='adaptive', knl=array([0., 0., 0., 0., 0., 0.]), ksl=array([0., 0., 0., 0., 0., 0.]), edge_entry_active=np.int64(1), edge_exit_active=np.int64(1), edge_entry_model='linear', edge_exit_model='linear', edge_entry_angle=0.00422, edge_exit_angle=0.00422, edge_entry_angle_fdown=0, edge_exit_angle_fdown=0, edge_entry_fint=0, edge_exit_fint=0, edge_entry_hgap=0, edge_exit_hgap=0, shift_x=0, shift_y=0, rot_s_rad=0)

In [ ]:
bend_elements = np.union1d(tt.rows['mba.*'].name, tt.rows['mbb.*'].name)

array(['mba.10030..0', 'mba.10030..0_aper_downstream',
       'mba.10030..0_aper_upstream', ..., 'mbb.63550_exit',
       'mbb.63550_exit_aper_downstream', 'mbb.63550_exit_aper_upstream'],
      shape=(13392,), dtype=object)